### Load Databricks using API
### 1. Install libraries


In [0]:
%pip install boxsdk pandas pyspark

### Step 1: Upload CSV files to Databricks

In [0]:
display(dbutils.fs.ls("dbfs:/FileStore/tables/"))


path,name,size,modificationTime
dbfs:/FileStore/tables/engagements_csv.zip,engagements_csv.zip,509212192,1741018054000
dbfs:/FileStore/tables/offers-1.csv,offers-1.csv,1761202,1741015687000
dbfs:/FileStore/tables/offers.csv,offers.csv,1761202,1741014507000
dbfs:/FileStore/tables/shopper_offers-1.csv,shopper_offers-1.csv,28568462,1741015732000
dbfs:/FileStore/tables/shopper_offers.csv,shopper_offers.csv,28568462,1741014524000
dbfs:/FileStore/tables/shoppers-1.csv,shoppers-1.csv,867248,1741015750000
dbfs:/FileStore/tables/shoppers.csv,shoppers.csv,867248,1741014509000


In [0]:
from pyspark.sql import SparkSession

# Initial Spark Session
spark = SparkSession.builder.appName ("CashbackUserBehaviour").getOrCreate()

# Define file path
engagements_path = "dbfs:/FileStore/tables/engagements_csv.zip"
offers_path = "dbfs:/FileStore/tables/offers.csv"
shopper_offers_path = "dbfs:/FileStore/tables/shopper_offers.csv"
shoppers_path = "dbfs:/FileStore/tables/shoppers.csv"

# Load datasets into Spark DataFrames
engagements_df = spark.read.csv(engagements_path, header = True, inferSchema = True)
offers_df = spark.read.csv(offers_path, header = True, inferSchema = True)
shopper_offers_df = spark.read.csv(shopper_offers_path, header = True, inferSchema = True)
shoppers_df = spark.read.csv(shoppers_path, header = True, inferSchema = True)

#Register datasets into Spark DataFrames
engagements_df.createOrReplaceTempView("engagements")
offers_df.createOrReplaceTempView("offers")
shopper_offers_df.createOrReplaceTempView("shopper_offers")
shoppers_df.createOrReplaceTempView("shoppers")

# Display schema to understand data structure
#engagements_df.show(5)
engagements_df.printSchema()
spark.sql("SHOW TABLES").show()

root
 |-- PK \b \b �zXY        �ը\   engagements.csvUT: string (nullable = true)

+--------+--------------+-----------+
|database|     tableName|isTemporary|
+--------+--------------+-----------+
|        |   engagements|       true|
|        |        offers|       true|
|        |shopper_offers|       true|
|        |      shoppers|       true|
+--------+--------------+-----------+



In [0]:
import os
import gdown

# Ensure directory exists
output_dir = '/dbfs/tmp/'
os.makedirs(output_dir, exist_ok = True)

# Google Drive file URL
url = 'https://drive.google.com/uc?export=download&id=1t7Vfus51OfpuZezfCiie3GxJWogeBOEN'

# Destination path in DBFS
output_path = os.path.join(output_dir, 'engagements.csv')

# Download the file
gdown.download(url, output_path, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1t7Vfus51OfpuZezfCiie3GxJWogeBOEN
From (redirected): https://drive.google.com/uc?export=download&id=1t7Vfus51OfpuZezfCiie3GxJWogeBOEN&confirm=t&uuid=36bc82a6-71ea-492d-8fa5-c651554d44aa
To: /dbfs/tmp/engagements.csv
100%|██████████| 1.55G/1.55G [00:12<00:00, 126MB/s]


Out[5]: '/dbfs/tmp/engagements.csv'

In [0]:
## Verify Column names in DataFrames
# Show schema of the shoppers dataset
shoppers_df.printSchema()

# Show first few rows to verify column names
shoppers_df.show(5)

# Show engagements dataset schema
engagements_df.printSchema()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-3915224358482294>:3
      1 ## Verify Column names in DataFrames
      2 # Show schema of the shoppers dataset
----> 3 shoppers_df.printSchema()
      5 # Show first few rows to verify column names
      6 shoppers_df.show(5)

NameError: name 'shoppers_df' is not defined

In [0]:
%sql
-- since Shoppers table contains 'customer_id' instead of 'shopper_id' we update accordingly

select s.customer_id, count(distinct e.event_type) as engagement_count
from shoppers s
left join engagements e on s.customer.id = e.customer_id
where e.event_timestamp between s.registration_date and date_add(s.registration_date, INTERVAL 6 MONTH)
group by s.customer_id;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3915224358482290>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-3915224358482290>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("c2VsZWN0IHMuc2hvcHBlcl9pZCwgY291bnQoZGlzdGluY3QgZS5ldmVudF90eXBlKSBhcyBlbmdhZ2VtZW50X2NvdW50CmZyb20gU2hvcHBlcnMgcwpsZWZ0IGpvaW4gRW5nYWdlbWVudCBlIG9uIHMuc2hvcHBlci5pZCA9IGUuc2hvcHBlcl9pZAp3aGVyZSBlLmV2ZW50X3RpbWVzdGFtcCBiZXR3ZWVuIHMucmVnaXN0cmF0aW9uX2RhdGUgYW5kIGRhdGVfYWRkKHMucmVnaXN0cmF0aW9uX2RhdGUsIElOVEVSVkFMIDYgTU9OVEgpCmdyb3VwIGJ5IHMuc2hvcHBlcl9pZA==").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:4

In [0]:
# Check / Verify Engagements Table Columns
engagements_df.printSchema()
engagements_df.show(5)

root
 |-- PK \b \b �zXY        �ը\   engagements.csvUT: string (nullable = true)

+------------------------------------------------+
|PK \b \b �zXY        �ը\   engagements.csvUT|
+------------------------------------------------+
|                             \a��g��g��gux...|
|                            1������'�R�9c...|
|                            �\ݑ:��V}�)�1�vF�...|
|                                        ����P<�|
|                            ���@-Nt!��9`\f+��...|
+------------------------------------------------+
only showing top 5 rows



In [0]:
# Check / Verify Shoppers Table Columns
shoppers_df.printSchema()
shoppers_df.show(5)

root
 |-- customer_id: long (nullable = true)
 |-- registration_date: date (nullable = true)
 |-- registration_device: string (nullable = true)

+-------------------+-----------------+-------------------+
|        customer_id|registration_date|registration_device|
+-------------------+-----------------+-------------------+
|6489927405216326396|       2018-01-01|                ios|
|3161625024140501939|       2018-01-01|                ios|
| 207638738633219587|       2018-01-01|                ios|
|3789491988850699117|       2018-01-03|                ios|
| 896562973335837221|       2018-01-03|                ios|
+-------------------+-----------------+-------------------+
only showing top 5 rows



In [0]:
# Verify that Engagements file is still zipped.
display(dbutils.fs.ls("dbfs:/FileStore/tables/"))

path,name,size,modificationTime
dbfs:/FileStore/tables/engagements_csv.zip,engagements_csv.zip,509212192,1741018054000
dbfs:/FileStore/tables/offers-1.csv,offers-1.csv,1761202,1741015687000
dbfs:/FileStore/tables/offers.csv,offers.csv,1761202,1741014507000
dbfs:/FileStore/tables/shopper_offers-1.csv,shopper_offers-1.csv,28568462,1741015732000
dbfs:/FileStore/tables/shopper_offers.csv,shopper_offers.csv,28568462,1741014524000
dbfs:/FileStore/tables/shoppers-1.csv,shoppers-1.csv,867248,1741015750000
dbfs:/FileStore/tables/shoppers.csv,shoppers.csv,867248,1741014509000


import zipfile

#### Define paths
zip_file_path = "/dbfs/FileStore/tables/engagements_csv.zip"  # Path to ZIP file
extract_to = "/dbfs/tmp/"  # Extract location

#### Extract ZIP file
with zipfile.ZipFile("/dbfs" + zip_file_path[4:], "r") as z:
    z.extractall(extract_to)  # Extract contents to tmp folder

#### List extracted files
display(dbutils.fs.ls("dbfs:/tmp/"))

In [0]:
%pip install gdown

Python interpreter will be restarted.
Python interpreter will be restarted.
